In [4]:
import matplotlib.pyplot as plt
import numpy as np
import cv2
import os
from PIL import Image
import matplotlib.pyplot as plt
import random
import pathlib
import pandas as pd
import torch
from torchvision import transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from torchvision.io import read_image
from torch.utils.data import Dataset
from torchvision import transforms
from utilss import get_accuracy, plot_curve, keep_store_dict, store_dict_to_disk
from data_loader import DataLoader1, ImageDataset
from initializers import gaussian_initializer, constant_initializer
from models import Model3, ModelM2, ModelM3, Model2, Model1, ModelAVE, Model
from test import test
from train import train

In [5]:
model1 = Model1()
model2 = Model2()
modelAVE = ModelAVE()
model2_M = ModelM2()
model3 = Model3()
model3_M = ModelM3()

In [6]:
#model = Model(model1, model2, model3, modelAVE, model2_M, model3_M)
model = Model1()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

In [7]:
num_epoch = 3
batch_size = 8
store_dict = dict()

In [8]:
dataset_test = ImageDataset("heat_maps_test_plain", "images_test", (368, 368), model.center_map)
dataset_training = ImageDataset("heat_maps_training_plain", "images_training", (368, 368), model.center_map)
dataset_valid = ImageDataset("heat_maps_valid_plain", "images_valid", (368, 368), model.center_map)


#test_data_loader = DataLoader(dataset_test, batch_size=batch_size, shuffle=True)


In [9]:
loss_function = nn.MSELoss()
learning_rate = 0.001
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [10]:
store_dict = train(model=model, num_epochs=num_epoch, train_loader=dataset_training, test_loader=dataset_valid, store_dict=store_dict, device = device, loss_function = loss_function, optimizer=optimizer, batch_size = batch_size)
store_dict_to_disk('store_dict.json', store_dict=store_dict)

1001


0it [00:00, ?it/s]/usr/local/lib/python3.11/dist-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(
0it [00:03, ?it/s]


ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [ ]:
torch.save(model,'/notebooks/lk-s-2024-detekcija-poze/model23_71.pth')

In [ ]:
loaded_model = torch.load('model23_7_2.pth')
loaded_model.eval()

input1 = dataset_training[5][0]
input1a = input1.cpu()
image_to_display = np.transpose(input1a, (1, 2, 0))
plt.imshow(image_to_display)
plt.show()
input1 = input1.to(device)
a = ['r ankle','r knee','r hip', 'l hipl', 'knee_X','l ankle_X','pelvis','thorax','upper neck','head top','r wrist','r elbow', 'r shoulder', 'l shoulder','l elbow','l wrist', 'back']
for i in range(17):
    output = loaded_model(input1)
    output = output.cpu()
    numpy_array = output.detach().numpy()
    plt.imshow(numpy_array[i]*255)
    plt.xlabel(a[i])
    plt.show()
   

In [ ]:
input1 = dataset_training[7060][0]
input1a = input1.cpu()
image_to_display = np.transpose(input1a, (1, 2, 0))
plt.imshow(image_to_display)
plt.show()